In [1]:
import numpy as np
import pandas as pd
import timeit
import time
import json
import re
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import normalize

import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

from word_match import regex_matcher
from document import survey_doc
from classify import classify_labels

In [2]:
docs = classify_labels()
print(len(docs.ul_df))
print(len(docs.l_df))

/home/matt_valley/anaconda3/envs/spacy/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


83060
123


In [3]:
docs.train_test(docs, model='tfidf')

83060 123
(83060, 1000) (123, 165)


ValueError: Found input variables with inconsistent numbers of samples: [83060, 123]

In [2]:
# load all data


paths = ['~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/AI_survey_data/2017 to mid 2018 comments.csv',
         '~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/AI_survey_data/2018 to mid 2019 comments.csv']

q1_df = pd.DataFrame()
q2_df = pd.DataFrame()
for data_path in paths:
    data = survey_doc(data_path)
    data.clean_unlabelled_data()
    data_q1 = data.df[(data.df['QID']=='61') | (data.df['QID']=='Unique / Unusual')]
    q1_df.append(data_q1, ignore_index=True)
    data_q2 = data.df[(data.df['QID']=='62') | (data.df['QID']=='One Change')]
    q2_df.append(data_q2, ignore_index=True)

/home/matt_valley/anaconda3/envs/spacy/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [3]:
# load labelled data
data_path = ['~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/AI_survey_data/Labeled data and sentiments 2017-18 data2.csv']
labeled_data = survey_doc(data_path[0], header=0)
labeled_data.clean_labelled_data()


Index(['Comment ID', 'Customer ID', 'Survey ID', 'Respondent ID', 'QID',
       'Comment', 'JK label', 'JK sentiment', 'Unnamed: 8'],
      dtype='object')


In [5]:
labeled_data.df.head()

,QID,text,JK label,JK sentiment
110,62,Clearer communication about expectations and s...,a posted schedule online would be incredibly ...,Support and resources


In [ ]:
survey_path_1 = '/home/matt_valley/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/AI_survey_data/2017 to mid 2018 comments.csv'
survey_path_2 = '/home/matt_valley/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/AI_survey_data/2018 to mid 2019 comments.csv'

df = pd.read_csv(survey_path_1, header=1)
df2 = pd.read_csv(survey_path_2, header=1)

print(len(df))
df = df.dropna(how='any')
df2 = df2.dropna(how='any')
print(len(df))

df = df.rename(columns={"Comment": "text"})
df2 = df2.rename(columns={"Comment": "text"})

text_clean = [len(t) for t in df.text.values]


# load spacy model

model = 'en_core_web_sm' # for testing on laptop
#model = 'en_core_web_lg'
#model = 'en_vectors_web_lg' # many more words
nlp = spacy.load(model)
#sentencizer = nlp.create_pipe("sentencizer")
#nlp.add_pipe(sentencizer)

In [ ]:
start_time = timeit.default_timer()

match = regex_matcher()
match_df = pd.DataFrame()
for t,text in enumerate(df['text']):
    doc = nlp(text)
    out_df = match.match_topics(t, doc)
    match_df = match_df.append(out_df)
    
match_df.reset_index(drop=True, inplace=True)

process_time = timeit.default_timer() - start_time
print(str(len(df)) + ' submissions, query took ' + str(process_time) + ' seconds')

In [ ]:
print(len(df))
print(len(match_df))
pd.to_pickle(match_df, '~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/match_df.pkl')

In [ ]:
match_df = pd.read_pickle('~/PycharmProjects/insight_2020a_project/Activated_Insights_consulting/match_df.pkl')

In [ ]:
### LOAD EMBEDDING MODEL

model = 'en_vectors_web_lg'
nlp_vec = spacy.load(model)

y_vec = []
topics = match_df['topic'].unique()
for topic in topics:
    y_doc = nlp(topic)
    y_vec.append(y_doc.vector)

In [ ]:
y_vec = np.array(y_vec)
y_vec.shape

In [ ]:
# BUILD CONTEXT VECTORS

X = []
for n,text in enumerate(match_df['context_span'].values):
    X.append(nlp(text).vector)
X = np.array(X)
print(X.shape)

In [ ]:
# dimensionality reduce

pca = PCA(n_components=20, whiten=True, random_state=42)
pca.fit(X, y)

In [ ]:
plt.plot(pca.explained_variance_)

In [ ]:
'''
do tfidf
'''

all_tfidf = []
topics = match_df['topic'].unique()
for topic in topics:
    temp_df = match_df[match_df['topic']==topic]
    data = [sent for sent in temp_df.comment_text]
    
    n_features = 1000
    n_components = 12
    n_top_words = 8
    
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                       max_features=n_features,
                                       stop_words='english')

    t0 = time.time()
    tfidf = tfidf_vectorizer.fit_transform(data)

    feature_array = np.array(tfidf_vectorizer.get_feature_names())
    tfidf_sorting = np.argsort(tfidf.toarray()).flatten()[::-1]
    
    n = 10
    top_n = feature_array[tfidf_sorting][:n]
    print(topic)
    print(top_n)
        
    all_tfidf.append(tfidf)
    print("done in %0.3fs." % (time.time() - t0))


In [ ]:
from scipy.sparse import csr_matrix
X = np.empty((0,all_tfidf[0].shape[1]), dtype=all_tfidf[0].dtype)
y = np.empty((0))
for t,topic_vecs in enumerate(all_tfidf):
    print(topic_vecs.shape)
    print(X.shape)
    X = np.append(X,topic_vecs.toarray(), axis=0)
    y = np.append(y,np.ones(topic_vecs.shape[0])*t, axis=0)
print(X.shape)
print(y.shape)


In [ ]:
logit = LogisticRegression(class_weight='balanced', 
                           random_state=42, 
                           multi_class='multinomial',
                           verbose=1,
                           max_iter=1000)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in skf.split(X, y):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    logit.fit(X_train, y_train)
    y_pred = logit.predict((X_test))
    
    print('accuracy = ' + str(metrics.accuracy_score(y_test, y_pred)))
    print('balenced accuracy = ' + str(metrics.balanced_accuracy_score(y_test, y_pred)))
    print('macro precision = ' + str(metrics.precision_score(y_logit.fit(X_train, y_train)

In [ ]:
y_pred = logit.predict((X_test))
print('accuracy = ' + str(metrics.accuracy_score(y_test, y_pred)))
print('balenced accuracy = ' + str(metrics.balanced_accuracy_score(y_test, y_pred)))
print('macro precision = ' + str(metrics.precision_score(y_test, y_pred,
                           average='macro')))
print('recall score = ' + str(metrics.recall_score(y_test, y_pred,
                           average='macro')))

In [ ]:
                               average='macro')))
    print('recall score = ' + str(metrics.recall_score(y_test, y_pred,
                               average='macro')))
    
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42,
                                                    stratify=y,
                                                    test_size=0.1)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)